In [10]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(
    api_wrapper=api_wrapper,
)

In [11]:
wiki.name

'wikipedia'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Artificial_intelligence")
docs = loader.load()
documents = text_splitter.split_documents(docs)
embeddings = OllamaEmbeddings(model="llama3.2")
vectorstore = FAISS.from_documents(documents, embeddings)   
retriver = vectorstore.as_retriever()

In [8]:
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F7E4BB3010>, search_kwargs={})

In [12]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriver,
    name="Artificial Intelligence Wikipedia Retriever",
    description="Useful for retrieving information from Wikipedia articles.",
)

In [13]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(
    api_wrapper=arxiv_api_wrapper,
)

In [14]:
arxiv.name

'arxiv'

In [15]:
tools = [arxiv,wiki, retriever_tool]

In [16]:
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Data Science Course\\Projects\\Langchain Tutorial\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Artificial Intelligence Wikipedia Retriever', description='Useful for retrieving information from Wikipedia articles.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001F7E49444A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langcha

In [19]:
from langchain_ollama import ChatOllama
chat = ChatOllama(model="llama3.2", temperature=0.1)

In [27]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent (chat,tools,prompt)

In [33]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor (agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [36]:
agent_executor.invoke({"input":"Tell me about Machine learning and its applications in Artificial Intelligence."})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning and its applications in Artificial Intelligence'}`


Page: Applications of artificial intelligence
Summary: Artificial intelligence (AI) has been used in applications throughout industry and academia. In a manner analogous to electricity or computers, AMachine Learning is a subset of Artificial Intelligence that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time.

Applications of Machine Learning in Artificial Intelligence include:

1. **Natural Language Processing (NLP)**: Machine learning is used to analyze and understand human language, enabling applications such as speech recognition, sentiment analysis, and language translation.
2. **Computer Vision**: Machine learning is used to enable computers to interpret and understand visual data from images and videos, enabling applications such as 

{'input': 'Tell me about Machine learning and its applications in Artificial Intelligence.',
 'output': 'Machine Learning is a subset of Artificial Intelligence that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time.\n\nApplications of Machine Learning in Artificial Intelligence include:\n\n1. **Natural Language Processing (NLP)**: Machine learning is used to analyze and understand human language, enabling applications such as speech recognition, sentiment analysis, and language translation.\n2. **Computer Vision**: Machine learning is used to enable computers to interpret and understand visual data from images and videos, enabling applications such as object detection, facial recognition, and image classification.\n3. **Predictive Maintenance**: Machine learning is used to predict when equipment is likely to fail, enabling proactive maintenance and reducing downtime.\n4. **Recommendation